## ToBigs 5주차

### Vision Advanced 과제

#### 문제 1.

Object Detection 에는 2-stage model 과 1-stage model이 존재합니다.  
각 유형에 해당하는 model을 하나씩 선정하여 설명하세요. (단, 세션 시간에 설명한 모델과 아래 제시된 모델은 제외할 것)

#### 문제 2.

아래 제시된 FasterRCNN 과 YOLOv5 를 각각 실행합니다.  
실행 결과를 제시하고 두 모델 사이의 차이점을 실행 결과에 근거하여 설명하세요.

In [17]:
## Package Import

import torch
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import CocoDetection
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import time
import warnings

warnings.filterwarnings("ignore")

In [18]:
## Data Download

import os
os.makedirs('./data/', exist_ok=True)
os.makedirs('./data/images/', exist_ok=True)

!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d ./data/
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d ./data/images/

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [19]:
import os
import urllib.request
import zipfile

# 데이터 디렉토리 생성
os.makedirs('./data/', exist_ok=True)
os.makedirs('./data/images/', exist_ok=True)

# 파일 다운로드 함수
def download_file(url, dest):
    print(f"Downloading {url} to {dest}...")
    urllib.request.urlretrieve(url, dest)
    print("Download complete!")

# 압축 해제 함수
def unzip_file(zip_path, extract_to):
    print(f"Unzipping {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("Unzip complete!")

# COCO 데이터셋 다운로드 URL
annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
images_url = "http://images.cocodataset.org/zips/val2017.zip"

# 다운로드 경로 설정
annotations_zip_path = './data/annotations_trainval2017.zip'
images_zip_path = './data/images/val2017.zip'

# 파일 다운로드 및 압축 해제
download_file(annotations_url, annotations_zip_path)
unzip_file(annotations_zip_path, './data/')
download_file(images_url, images_zip_path)
unzip_file(images_zip_path, './data/images/')


Download complete!
Unzipping ./data/annotations_trainval2017.zip...
Unzip complete!
Download complete!
Unzipping ./data/images/val2017.zip...
Unzip complete!


In [25]:
## FasterRCNN

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

# 모델 로드 및 설정
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 모델의 클래스 수를 3개로 설정 (배경 포함해서 0, 2, background)
num_classes = 3  # COCO의 경우 background 포함 (0, 2 두 개의 클래스 + 배경)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 모델을 평가 모드로 설정
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = list(image.to(device) for image in images)

        # 각 이미지의 타깃을 적절히 변환하여 GPU로 전송
        processed_targets = []
        for target in targets:
            processed_target = {}
            processed_target['boxes'] = torch.tensor([ann['bbox'] for ann in target]).to(device)
            processed_target['labels'] = torch.tensor([ann['category_id'] for ann in target]).to(device)
            processed_targets.append(processed_target)

        # 모델 예측
        outputs = model(images)

        for i, output in enumerate(outputs):
            pred_labels = output['labels'].cpu().numpy()
            true_labels = processed_targets[i]['labels'].cpu().numpy()

            # 예측 수가 실제 라벨 수보다 많은 경우 예측 수를 잘라냄
            if len(pred_labels) > len(true_labels):
                pred_labels = pred_labels[:len(true_labels)]

            # 예측 레이블과 실제 레이블이 얼마나 일치하는지 확인
            correct = np.sum(pred_labels == true_labels[:len(pred_labels)])
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time
accuracy = total_correct / total_predictions if total_predictions > 0 else 0

print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy * 100:.2f}%")

loading annotations into memory...
Done (t=1.08s)
creating index...
index created!


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\이호/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|███████████████████████████████████████████████████████████████████████████████| 160M/160M [00:02<00:00, 68.3MB/s]


PicklingError: Can't pickle <function <lambda> at 0x000001F2833F8C10>: attribute lookup <lambda> on __main__ failed

In [ ]:
## YOLOv5

# 경로 설정
image_dir = "./data/images/val2017/"
json_path = "./data/annotations/instances_val2017.json"

# Transform 설정
transform = transforms.Compose([
    transforms.Resize((640, 640)),  # YOLOv5의 기본 입력 크기
    transforms.ToTensor()
])

# Dataset과 DataLoader 설정
dataset = CocoDetection(root=image_dir, annFile=json_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

# 모델 로드 (PyTorch Hub에서 COCO로 사전 학습된 YOLOv5 모델 로드)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# GPU 사용 가능 시 GPU로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

# 정확도 계산을 위한 변수
total_correct = 0
total_predictions = 0

# 소요 시간 측정 시작
start_time = time.time()

# 100장 예측 속도를 비교
cnt = 0

with torch.no_grad():
    for images, targets in data_loader:
        cnt += 1
        images = images.to(device)

        # 모델 예측
        outputs = model(images)

        # 예측된 바운딩 박스와 클래스 정보 추출
        for output in outputs:
            pred_labels = output[:, 5:].argmax(1).cpu().numpy()  # 예측된 클래스 레이블

            # 실제 라벨을 추출하고, 예측된 라벨과 비교
            true_labels = [t['category_id'].item() for t in targets]

            # 예측된 라벨과 실제 라벨이 일치하는지 확인
            correct = np.sum(pred_labels[:len(true_labels)] == true_labels)
            total_correct += correct
            total_predictions += len(true_labels)

        if cnt == 100:
            break

# 소요 시간 측정 종료
end_time = time.time()

# 소요 시간 및 정확도 출력
time_taken = end_time - start_time

# 정확도 계산
if total_predictions > 0:
    accuracy = (total_correct / total_predictions) * 100
else:
    accuracy = 0

# 소요 시간 및 정확도 출력
print(f"소요 시간: {time_taken:.2f}초")
print(f"정확도: {accuracy:.2f}%")